This notebook generates a table of intensities and colours for the visualiation of raw timsTOF data, and generates a JSON lookup file.

In [21]:
from matplotlib import colors, cm, pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image

In [22]:
MINIMUM_PIXEL_INTENSITY = 1
MAXIMUM_PIXEL_INTENSITY = 1000

PIXELS_X = 1000
PIXELS_Y = 30

In [23]:
# create the colour map to convert intensity to colour
colour_map = plt.get_cmap('rainbow')
norm = colors.LogNorm(vmin=MINIMUM_PIXEL_INTENSITY, vmax=MAXIMUM_PIXEL_INTENSITY, clip=True)  # aiming to get good colour variation in the lower range, and clipping everything else

In [24]:
colours_l = []
for i in range(MINIMUM_PIXEL_INTENSITY,MAXIMUM_PIXEL_INTENSITY+1):
    colours_l.append((i, colour_map(norm(i), bytes=True)[:3]))
colours_df = pd.DataFrame(colours_l, columns=['intensity','colour'])

In [25]:
im_array = np.zeros([PIXELS_Y+1, PIXELS_X+1, 3], dtype=np.uint8)

In [26]:
for r in zip(colours_df.intensity, colours_df.colour):
    x = r[0]
    im_array[:,x,:] = r[1]


In [27]:
im = Image.fromarray(im_array, 'RGB')


In [28]:
im.show()

In [31]:
# write out the colour table to a JSON file
colourmap_file_name = '/Users/darylwilding-mcbride/Downloads/colourmap.json'
colours_df.to_json(colourmap_file_name, orient='records')